# MoK Target Variable Visualization

This notebook provides comprehensive visualizations and statistical analysis of the MoK (Monsoon Onset over Kerala) target variable: **DateRelJun01** (days relative to June 1st).

## Dataset Information
- **Target Variable**: DateRelJun01 - Days relative to June 1st
- **Time Range**: 1940-2025
- **Negative values**: Monsoon onset before June 1st
- **Positive values**: Monsoon onset after June 1st
- **Zero**: Monsoon onset exactly on June 1st

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## 1. Load Data

In [ ]:
# Load the MoK dates data
data_path = '/home/deepakns/Work/data/MoKDates.csv'
df = pd.read_csv(data_path)

# Display basic info
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nFirst few rows:")
df.head(10)

In [ ]:
# Convert DateRelJun01 to actual dates for better interpretation
def rel_days_to_date(year, days_rel):
    """Convert days relative to June 1st to actual date"""
    june_1 = datetime(year, 6, 1)
    actual_date = june_1 + timedelta(days=days_rel)
    return actual_date

df['ActualDate'] = df.apply(lambda row: rel_days_to_date(int(row['Year']), row['DateRelJun01']), axis=1)
df['Month'] = df['ActualDate'].dt.month
df['Day'] = df['ActualDate'].dt.day
df['MonthName'] = df['ActualDate'].dt.strftime('%B')
df['DateString'] = df['ActualDate'].dt.strftime('%B %d')

print("Sample data with converted dates:")
df[['Year', 'DateRelJun01', 'ActualDate', 'DateString']].head(10)

## 2. Basic Statistics

In [ ]:
# Descriptive statistics
print("="*60)
print("DESCRIPTIVE STATISTICS FOR DateRelJun01")
print("="*60)
print(df['DateRelJun01'].describe())
print(f"\nSkewness: {df['DateRelJun01'].skew():.4f}")
print(f"Kurtosis: {df['DateRelJun01'].kurtosis():.4f}")
print(f"\nMode: {df['DateRelJun01'].mode().values}")
print(f"Median: {df['DateRelJun01'].median():.2f}")
print(f"Mean: {df['DateRelJun01'].mean():.2f}")

In [ ]:
# Distribution of onset timing
print("\n" + "="*60)
print("ONSET TIMING DISTRIBUTION")
print("="*60)
early_onset = (df['DateRelJun01'] < 0).sum()
on_time = (df['DateRelJun01'] == 0).sum()
late_onset = (df['DateRelJun01'] > 0).sum()

print(f"Early onset (before June 1): {early_onset} ({early_onset/len(df)*100:.1f}%)")
print(f"On-time onset (June 1): {on_time} ({on_time/len(df)*100:.1f}%)")
print(f"Late onset (after June 1): {late_onset} ({late_onset/len(df)*100:.1f}%)")

print(f"\nEarliest onset: {df['DateRelJun01'].min():.0f} days (Year: {df.loc[df['DateRelJun01'].idxmin(), 'Year']:.0f}) - {df.loc[df['DateRelJun01'].idxmin(), 'DateString']}")
print(f"Latest onset: {df['DateRelJun01'].max():.0f} days (Year: {df.loc[df['DateRelJun01'].idxmax(), 'Year']:.0f}) - {df.loc[df['DateRelJun01'].idxmax(), 'DateString']}")
print(f"Range: {df['DateRelJun01'].max() - df['DateRelJun01'].min():.0f} days")

## 3. Time Series Visualization

In [ ]:
# Time series plot with train/val/test splits
fig, ax = plt.subplots(figsize=(16, 6))

# Define splits based on config
train_mask = (df['Year'] >= 1951) & (df['Year'] <= 2000)
val_mask = (df['Year'] >= 2001) & (df['Year'] <= 2010)
test_mask = (df['Year'] >= 2011) & (df['Year'] <= 2024)

# Plot data
ax.plot(df[train_mask]['Year'], df[train_mask]['DateRelJun01'], 'o-', label='Train (1951-2000)', alpha=0.7, linewidth=2)
ax.plot(df[val_mask]['Year'], df[val_mask]['DateRelJun01'], 's-', label='Validation (2001-2010)', alpha=0.7, linewidth=2)
ax.plot(df[test_mask]['Year'], df[test_mask]['DateRelJun01'], '^-', label='Test (2011-2024)', alpha=0.7, linewidth=2)

# Add reference lines
ax.axhline(y=0, color='red', linestyle='--', alpha=0.5, label='June 1st')
ax.axhline(y=df['DateRelJun01'].mean(), color='green', linestyle='--', alpha=0.5, label=f'Mean ({df["DateRelJun01"].mean():.1f} days)')

# Styling
ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Days Relative to June 1st', fontsize=12, fontweight='bold')
ax.set_title('MoK Onset Date Time Series (1940-2025)', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Time series with rolling statistics
fig, ax = plt.subplots(figsize=(16, 6))

# Calculate rolling statistics
window = 10
rolling_mean = df['DateRelJun01'].rolling(window=window, center=True).mean()
rolling_std = df['DateRelJun01'].rolling(window=window, center=True).std()

# Plot
ax.plot(df['Year'], df['DateRelJun01'], 'o-', alpha=0.4, label='Actual', markersize=4)
ax.plot(df['Year'], rolling_mean, 'r-', linewidth=2.5, label=f'{window}-year Rolling Mean')
ax.fill_between(df['Year'], 
                rolling_mean - rolling_std, 
                rolling_mean + rolling_std, 
                alpha=0.2, label=f'{window}-year Rolling Std Dev')

ax.axhline(y=0, color='black', linestyle='--', alpha=0.5, label='June 1st')

ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Days Relative to June 1st', fontsize=12, fontweight='bold')
ax.set_title(f'MoK Onset with {window}-Year Rolling Statistics', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Distribution Analysis

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histogram with KDE
axes[0, 0].hist(df['DateRelJun01'], bins=30, edgecolor='black', alpha=0.7, density=True)
df['DateRelJun01'].plot(kind='kde', ax=axes[0, 0], linewidth=2, color='red')
axes[0, 0].axvline(df['DateRelJun01'].mean(), color='green', linestyle='--', linewidth=2, label=f'Mean: {df["DateRelJun01"].mean():.1f}')
axes[0, 0].axvline(df['DateRelJun01'].median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {df["DateRelJun01"].median():.1f}')
axes[0, 0].set_xlabel('Days Relative to June 1st', fontweight='bold')
axes[0, 0].set_ylabel('Density', fontweight='bold')
axes[0, 0].set_title('Distribution with KDE', fontweight='bold', fontsize=12)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Box plot
bp = axes[0, 1].boxplot(df['DateRelJun01'], vert=True, patch_artist=True,
                        boxprops=dict(facecolor='lightblue', alpha=0.7),
                        medianprops=dict(color='red', linewidth=2),
                        whiskerprops=dict(linewidth=1.5),
                        capprops=dict(linewidth=1.5))
axes[0, 1].set_ylabel('Days Relative to June 1st', fontweight='bold')
axes[0, 1].set_title('Box Plot', fontweight='bold', fontsize=12)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Add statistics to box plot
stats_text = f"Min: {df['DateRelJun01'].min():.0f}\nQ1: {df['DateRelJun01'].quantile(0.25):.0f}\nMedian: {df['DateRelJun01'].median():.0f}\nQ3: {df['DateRelJun01'].quantile(0.75):.0f}\nMax: {df['DateRelJun01'].max():.0f}"
axes[0, 1].text(1.3, df['DateRelJun01'].median(), stats_text, fontsize=9, verticalalignment='center')

# Violin plot
parts = axes[1, 0].violinplot([df['DateRelJun01']], positions=[0], showmeans=True, showmedians=True)
axes[1, 0].set_ylabel('Days Relative to June 1st', fontweight='bold')
axes[1, 0].set_title('Violin Plot', fontweight='bold', fontsize=12)
axes[1, 0].set_xticks([0])
axes[1, 0].set_xticklabels(['DateRelJun01'])
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Q-Q plot for normality check
stats.probplot(df['DateRelJun01'], dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot (Normality Test)', fontweight='bold', fontsize=12)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Normality test
from scipy.stats import shapiro, normaltest

print("="*60)
print("NORMALITY TESTS")
print("="*60)

# Shapiro-Wilk test
shapiro_stat, shapiro_p = shapiro(df['DateRelJun01'])
print(f"\nShapiro-Wilk Test:")
print(f"  Test Statistic: {shapiro_stat:.4f}")
print(f"  P-value: {shapiro_p:.4f}")
print(f"  Result: {'Normal distribution' if shapiro_p > 0.05 else 'Not normal distribution'} (α=0.05)")

# D'Agostino's K-squared test
normal_stat, normal_p = normaltest(df['DateRelJun01'])
print(f"\nD'Agostino's K-squared Test:")
print(f"  Test Statistic: {normal_stat:.4f}")
print(f"  P-value: {normal_p:.4f}")
print(f"  Result: {'Normal distribution' if normal_p > 0.05 else 'Not normal distribution'} (α=0.05)")

## 5. Frequency and Categorical Analysis

In [ ]:
# Month-wise distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count by month
month_counts = df['MonthName'].value_counts().sort_index()
month_order = ['May', 'June', 'July']
month_counts = month_counts.reindex(month_order, fill_value=0)

axes[0].bar(month_counts.index, month_counts.values, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Month', fontweight='bold')
axes[0].set_ylabel('Frequency', fontweight='bold')
axes[0].set_title('Onset Frequency by Month', fontweight='bold', fontsize=12)
axes[0].grid(True, alpha=0.3, axis='y')

# Add counts on bars
for i, (month, count) in enumerate(month_counts.items()):
    axes[0].text(i, count + 0.5, str(int(count)), ha='center', fontweight='bold')

# Onset category pie chart
categories = ['Early\n(Before Jun 1)', 'On-time\n(Jun 1)', 'Late\n(After Jun 1)']
values = [early_onset, on_time, late_onset]
colors = ['#ff9999', '#66b3ff', '#99ff99']
explode = (0.05, 0.05, 0.05)

axes[1].pie(values, labels=categories, autopct='%1.1f%%', startangle=90, 
            colors=colors, explode=explode, textprops={'fontsize': 11, 'fontweight': 'bold'})
axes[1].set_title('Onset Timing Categories', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Value counts and frequency table
value_counts = df['DateRelJun01'].value_counts().sort_index()
print("\n" + "="*60)
print("MOST COMMON ONSET DATES")
print("="*60)
top_dates = df['DateRelJun01'].value_counts().head(10)
for days, count in top_dates.items():
    date_str = df[df['DateRelJun01'] == days]['DateString'].iloc[0]
    print(f"  {days:3.0f} days ({date_str:15s}): {count} occurrences")

## 6. Temporal Patterns

In [ ]:
# Decade-wise analysis
df['Decade'] = (df['Year'] // 10) * 10

fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Box plot by decade
decade_data = [df[df['Decade'] == d]['DateRelJun01'].values for d in sorted(df['Decade'].unique())]
decade_labels = [f"{int(d)}s" for d in sorted(df['Decade'].unique())]

bp = axes[0].boxplot(decade_data, labels=decade_labels, patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
    patch.set_alpha(0.7)

axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5, label='June 1st')
axes[0].set_xlabel('Decade', fontweight='bold')
axes[0].set_ylabel('Days Relative to June 1st', fontweight='bold')
axes[0].set_title('Onset Date Distribution by Decade', fontweight='bold', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Bar plot of decade means
decade_stats = df.groupby('Decade')['DateRelJun01'].agg(['mean', 'std']).reset_index()
x_pos = range(len(decade_stats))

axes[1].bar(x_pos, decade_stats['mean'], yerr=decade_stats['std'], 
            capsize=5, alpha=0.7, edgecolor='black')
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5, label='June 1st')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(decade_labels)
axes[1].set_xlabel('Decade', fontweight='bold')
axes[1].set_ylabel('Mean Days Relative to June 1st', fontweight='bold')
axes[1].set_title('Mean Onset Date by Decade (with Std Dev)', fontweight='bold', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Decade-wise statistics table
print("\n" + "="*80)
print("DECADE-WISE STATISTICS")
print("="*80)
decade_summary = df.groupby('Decade')['DateRelJun01'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Std Dev', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
]).round(2)
decade_summary.index = [f"{int(d)}s" for d in decade_summary.index]
print(decade_summary)

## 7. Trend Analysis

In [ ]:
# Linear trend analysis
from scipy.stats import linregress

# Perform linear regression
slope, intercept, r_value, p_value, std_err = linregress(df['Year'], df['DateRelJun01'])

# Generate trend line
trend_line = slope * df['Year'] + intercept

# Plot
fig, ax = plt.subplots(figsize=(16, 6))

ax.scatter(df['Year'], df['DateRelJun01'], alpha=0.6, s=50, label='Actual Data')
ax.plot(df['Year'], trend_line, 'r-', linewidth=2.5, label=f'Linear Trend (slope={slope:.4f})')
ax.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='June 1st')

# Add equation and statistics
equation_text = f"y = {slope:.4f}x + {intercept:.2f}\nR² = {r_value**2:.4f}\np-value = {p_value:.4f}"
ax.text(0.02, 0.98, equation_text, transform=ax.transAxes, 
        fontsize=11, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Days Relative to June 1st', fontsize=12, fontweight='bold')
ax.set_title('Linear Trend Analysis of MoK Onset Dates', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print interpretation
print("\n" + "="*60)
print("TREND ANALYSIS RESULTS")
print("="*60)
print(f"Slope: {slope:.6f} days/year")
print(f"R-squared: {r_value**2:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Standard Error: {std_err:.6f}")

if p_value < 0.05:
    trend_direction = "delayed" if slope > 0 else "earlier"
    print(f"\n✓ Significant trend detected: Monsoon onset is becoming {trend_direction} over time")
else:
    print(f"\n✗ No significant trend detected (p > 0.05)")

# Calculate change over the period
years_span = df['Year'].max() - df['Year'].min()
total_change = slope * years_span
print(f"\nEstimated change from {df['Year'].min():.0f} to {df['Year'].max():.0f}: {total_change:.2f} days")

## 8. Autocorrelation Analysis

In [ ]:
# Autocorrelation and Partial Autocorrelation
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Autocorrelation Function (ACF)
plot_acf(df['DateRelJun01'], lags=20, ax=axes[0], alpha=0.05)
axes[0].set_title('Autocorrelation Function (ACF)', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Lag', fontweight='bold')
axes[0].set_ylabel('Autocorrelation', fontweight='bold')

# Partial Autocorrelation Function (PACF)
plot_pacf(df['DateRelJun01'], lags=20, ax=axes[1], alpha=0.05)
axes[1].set_title('Partial Autocorrelation Function (PACF)', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Lag', fontweight='bold')
axes[1].set_ylabel('Partial Autocorrelation', fontweight='bold')

plt.tight_layout()
plt.show()

## 9. Split-wise Analysis

In [ ]:
# Compare statistics across train/val/test splits
df['Split'] = 'Other'
df.loc[train_mask, 'Split'] = 'Train'
df.loc[val_mask, 'Split'] = 'Validation'
df.loc[test_mask, 'Split'] = 'Test'

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Box plot by split
split_order = ['Train', 'Validation', 'Test']
split_data = [df[df['Split'] == s]['DateRelJun01'].values for s in split_order]

bp = axes[0].boxplot(split_data, labels=split_order, patch_artist=True)
colors = ['#ff9999', '#66b3ff', '#99ff99']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5, label='June 1st')
axes[0].set_ylabel('Days Relative to June 1st', fontweight='bold')
axes[0].set_title('Distribution by Data Split', fontweight='bold', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Bar plot of means
split_stats = df[df['Split'].isin(split_order)].groupby('Split')['DateRelJun01'].agg(['mean', 'std'])
split_stats = split_stats.reindex(split_order)

x_pos = range(len(split_stats))
axes[1].bar(x_pos, split_stats['mean'], yerr=split_stats['std'], 
            capsize=5, alpha=0.7, edgecolor='black', color=colors)
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5, label='June 1st')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(split_order)
axes[1].set_ylabel('Mean Days Relative to June 1st', fontweight='bold')
axes[1].set_title('Mean Onset Date by Split (with Std Dev)', fontweight='bold', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (mean, std) in enumerate(zip(split_stats['mean'], split_stats['std'])):
    axes[1].text(i, mean + std + 0.5, f'{mean:.1f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Split-wise statistics table
print("\n" + "="*80)
print("SPLIT-WISE STATISTICS")
print("="*80)
split_summary = df[df['Split'].isin(split_order)].groupby('Split')['DateRelJun01'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Std Dev', 'std'),
    ('Min', 'min'),
    ('Max', 'max'),
    ('Range', lambda x: x.max() - x.min())
]).round(2)
split_summary = split_summary.reindex(split_order)
print(split_summary)

# Check for distribution shift
from scipy.stats import ks_2samp
print("\n" + "="*80)
print("DISTRIBUTION SHIFT TESTS (Kolmogorov-Smirnov)")
print("="*80)

train_data = df[train_mask]['DateRelJun01']
val_data = df[val_mask]['DateRelJun01']
test_data = df[test_mask]['DateRelJun01']

ks_stat_tv, ks_p_tv = ks_2samp(train_data, val_data)
ks_stat_tt, ks_p_tt = ks_2samp(train_data, test_data)

print(f"\nTrain vs Validation:")
print(f"  KS Statistic: {ks_stat_tv:.4f}")
print(f"  P-value: {ks_p_tv:.4f}")
print(f"  Result: {'Significantly different' if ks_p_tv < 0.05 else 'Not significantly different'} (α=0.05)")

print(f"\nTrain vs Test:")
print(f"  KS Statistic: {ks_stat_tt:.4f}")
print(f"  P-value: {ks_p_tt:.4f}")
print(f"  Result: {'Significantly different' if ks_p_tt < 0.05 else 'Not significantly different'} (α=0.05)")

## 10. Outlier Analysis

In [ ]:
# Identify outliers using IQR method
Q1 = df['DateRelJun01'].quantile(0.25)
Q3 = df['DateRelJun01'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['DateRelJun01'] < lower_bound) | (df['DateRelJun01'] > upper_bound)]

print("="*60)
print("OUTLIER ANALYSIS (IQR Method)")
print("="*60)
print(f"Q1: {Q1:.2f}")
print(f"Q3: {Q3:.2f}")
print(f"IQR: {IQR:.2f}")
print(f"Lower Bound: {lower_bound:.2f}")
print(f"Upper Bound: {upper_bound:.2f}")
print(f"\nNumber of outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")

if len(outliers) > 0:
    print(f"\nOutlier years and values:")
    for _, row in outliers.sort_values('DateRelJun01').iterrows():
        print(f"  {row['Year']:.0f}: {row['DateRelJun01']:6.0f} days ({row['DateString']})")

In [ ]:
# Visualize outliers
fig, ax = plt.subplots(figsize=(16, 6))

# Plot all data
ax.scatter(df['Year'], df['DateRelJun01'], alpha=0.5, s=50, label='Normal')

# Highlight outliers
if len(outliers) > 0:
    ax.scatter(outliers['Year'], outliers['DateRelJun01'], 
              color='red', s=100, marker='X', label='Outliers', zorder=5)
    
    # Annotate outliers
    for _, row in outliers.iterrows():
        ax.annotate(f"{row['Year']:.0f}\n{row['DateRelJun01']:.0f}d", 
                   xy=(row['Year'], row['DateRelJun01']),
                   xytext=(10, 10), textcoords='offset points',
                   fontsize=8, bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                   arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

# Add bounds
ax.axhline(y=lower_bound, color='orange', linestyle='--', alpha=0.5, label=f'Lower Bound ({lower_bound:.1f})')
ax.axhline(y=upper_bound, color='orange', linestyle='--', alpha=0.5, label=f'Upper Bound ({upper_bound:.1f})')
ax.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='June 1st')

ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Days Relative to June 1st', fontsize=12, fontweight='bold')
ax.set_title('Outlier Detection in MoK Onset Dates', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Summary and Key Insights

In [ ]:
print("="*80)
print("SUMMARY OF KEY INSIGHTS")
print("="*80)

print(f"\n1. DATA OVERVIEW")
print(f"   • Total observations: {len(df)}")
print(f"   • Time span: {df['Year'].min():.0f} - {df['Year'].max():.0f}")
print(f"   • Date range: {df['DateString'].iloc[df['DateRelJun01'].idxmin()]} to {df['DateString'].iloc[df['DateRelJun01'].idxmax()]}")

print(f"\n2. CENTRAL TENDENCY")
print(f"   • Mean: {df['DateRelJun01'].mean():.2f} days relative to June 1st")
print(f"   • Median: {df['DateRelJun01'].median():.2f} days")
print(f"   • Mode: {df['DateRelJun01'].mode().values[0]:.0f} days")

print(f"\n3. VARIABILITY")
print(f"   • Standard deviation: {df['DateRelJun01'].std():.2f} days")
print(f"   • Range: {df['DateRelJun01'].max() - df['DateRelJun01'].min():.0f} days")
print(f"   • IQR: {IQR:.2f} days")

print(f"\n4. DISTRIBUTION SHAPE")
print(f"   • Skewness: {df['DateRelJun01'].skew():.4f} ({'right-skewed' if df['DateRelJun01'].skew() > 0 else 'left-skewed'})")
print(f"   • Kurtosis: {df['DateRelJun01'].kurtosis():.4f} ({'leptokurtic (heavy tails)' if df['DateRelJun01'].kurtosis() > 0 else 'platykurtic (light tails)'})")
print(f"   • Normality: {'Approximately normal' if shapiro_p > 0.05 else 'Not normal'} (Shapiro-Wilk p={shapiro_p:.4f})")

print(f"\n5. TEMPORAL TRENDS")
trend_direction = "delayed" if slope > 0 else "earlier"
trend_significance = "significant" if p_value < 0.05 else "not significant"
print(f"   • Linear trend: {slope:.6f} days/year ({trend_direction})")
print(f"   • Trend significance: {trend_significance} (p={p_value:.4f})")
print(f"   • Total change ({df['Year'].min():.0f}-{df['Year'].max():.0f}): {total_change:.2f} days")

print(f"\n6. ONSET TIMING CATEGORIES")
print(f"   • Early (before Jun 1): {early_onset} years ({early_onset/len(df)*100:.1f}%)")
print(f"   • On-time (Jun 1): {on_time} years ({on_time/len(df)*100:.1f}%)")
print(f"   • Late (after Jun 1): {late_onset} years ({late_onset/len(df)*100:.1f}%)")

print(f"\n7. OUTLIERS")
print(f"   • Number of outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")
if len(outliers) > 0:
    extreme_early = outliers.loc[outliers['DateRelJun01'].idxmin()]
    extreme_late = outliers.loc[outliers['DateRelJun01'].idxmax()]
    print(f"   • Most extreme early: {extreme_early['Year']:.0f} ({extreme_early['DateRelJun01']:.0f} days, {extreme_early['DateString']})")
    print(f"   • Most extreme late: {extreme_late['Year']:.0f} ({extreme_late['DateRelJun01']:.0f} days, {extreme_late['DateString']})")

print(f"\n8. DATA SPLITS")
for split in split_order:
    split_df = df[df['Split'] == split]
    print(f"   • {split}: {len(split_df)} samples, mean={split_df['DateRelJun01'].mean():.2f}d, std={split_df['DateRelJun01'].std():.2f}d")

print(f"\n9. DISTRIBUTION SHIFT")
print(f"   • Train vs Validation: {'Significantly different' if ks_p_tv < 0.05 else 'Similar'} (KS p={ks_p_tv:.4f})")
print(f"   • Train vs Test: {'Significantly different' if ks_p_tt < 0.05 else 'Similar'} (KS p={ks_p_tt:.4f})")

print("\n" + "="*80)

## Conclusions

This notebook provides a comprehensive analysis of the MoK onset dates from 1940-2025. Key findings:

1. **Target Variable Range**: The onset dates vary significantly, with extremes from May to July
2. **Distribution**: The data shows specific characteristics in terms of skewness and kurtosis
3. **Temporal Patterns**: Analysis reveals any trends or patterns over the decades
4. **Data Quality**: Outlier analysis helps identify unusual years
5. **Model Implications**: Split-wise analysis ensures fair train/validation/test distribution

These visualizations and statistics provide valuable context for:
- Model training and evaluation
- Understanding prediction challenges
- Identifying potential biases or shifts in the data
- Setting realistic performance expectations

## 12. Binned Analysis of Onset Dates

In this section, we create meaningful bins for onset dates to better understand the distribution patterns:
- **On or Before May 20**: Very early onset (≤ May 20)
- **May 21-24**: Early onset
- **May 25-27**: Moderately early
- **May 28-30**: Slightly early
- **May 31-June 2**: Normal/on-time
- **June 3-5**: Slightly late
- **June 6-8**: Moderately late
- **June 9-12**: Late onset
- **After June 12**: Very late onset

In [ ]:
# Define bins based on days relative to June 1st
# May 20 = -12 days, May 21 = -11 days, May 24 = -8 days, May 25 = -7 days
# May 27 = -5 days, May 28 = -4 days, May 30 = -2 days, May 31 = -1 day
# June 1 = 0 days, June 2 = +1 day, June 3 = +2 days, June 5 = +4 days
# June 6 = +5 days, June 8 = +7 days, June 9 = +8 days, June 12 = +11 days

# Using pandas.cut with right=True (default): bins are (lower, upper]
# So we need edges that create: ≤-12, (-12,-8], (-8,-5], (-5,-2], (-2,+1], (+1,+4], (+4,+7], (+7,+11], >+11
bin_edges = [-np.inf, -12, -8, -5, -2, 1, 4, 7, 11, np.inf]
bin_labels = [
    'On/Before May 20\n(≤ -12d)',
    'May 21-24\n(-12 to -9d)',
    'May 25-27\n(-8 to -6d)',
    'May 28-30\n(-5 to -3d)',
    'May 31-Jun 2\n(-2 to +1d)',
    'June 3-5\n(+2 to +4d)',
    'June 6-8\n(+5 to +7d)',
    'June 9-12\n(+8 to +11d)',
    'After June 12\n(> +11d)'
]

# Create binned categories
df['OnsetBin'] = pd.cut(df['DateRelJun01'], bins=bin_edges, labels=bin_labels, include_lowest=True)

# Display bin distribution
print("="*80)
print("BINNED ONSET DATE DISTRIBUTION")
print("="*80)
bin_counts = df['OnsetBin'].value_counts().sort_index()
print("\nFrequency by bin:")
for bin_label, count in bin_counts.items():
    percentage = count / len(df) * 100
    print(f"  {bin_label:25s}: {count:3d} occurrences ({percentage:5.1f}%)")

print(f"\nTotal: {len(df)} observations")

In [ ]:
# Overall histogram of binned data
fig, ax = plt.subplots(figsize=(16, 8))

bin_counts_ordered = df['OnsetBin'].value_counts().reindex(bin_labels, fill_value=0)

# Create color gradient
colors_palette = plt.cm.RdYlGn_r(np.linspace(0.1, 0.9, len(bin_labels)))

bars = ax.bar(range(len(bin_labels)), bin_counts_ordered.values, 
               color=colors_palette, edgecolor='black', linewidth=1.5, alpha=0.8)

# Add value labels on bars
for i, (bar, count) in enumerate(zip(bars, bin_counts_ordered.values)):
    height = bar.get_height()
    percentage = count / len(df) * 100
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.5,
            f'{int(count)}\n({percentage:.1f}%)',
            ha='center', va='bottom', fontweight='bold', fontsize=10)

ax.set_xticks(range(len(bin_labels)))
ax.set_xticklabels(bin_labels, rotation=45, ha='right', fontsize=10)
ax.set_xlabel('Onset Date Bins', fontsize=12, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax.set_title('Distribution of MoK Onset Dates by Bins (All Years)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 12.1 Binned Distribution by Decade

In [ ]:
# Binned distribution by decade
decades = sorted(df['Decade'].unique())
decade_labels_list = [f"{int(d)}s" for d in decades]

# Create a cross-tabulation
decade_bin_crosstab = pd.crosstab(df['Decade'], df['OnsetBin'])
decade_bin_crosstab = decade_bin_crosstab.reindex(columns=bin_labels, fill_value=0)
decade_bin_crosstab.index = decade_labels_list

print("="*100)
print("ONSET DATE BIN DISTRIBUTION BY DECADE (Count)")
print("="*100)
print(decade_bin_crosstab)

# Percentage within each decade
decade_bin_pct = pd.crosstab(df['Decade'], df['OnsetBin'], normalize='index') * 100
decade_bin_pct = decade_bin_pct.reindex(columns=bin_labels, fill_value=0)
decade_bin_pct.index = decade_labels_list

print("\n" + "="*100)
print("ONSET DATE BIN DISTRIBUTION BY DECADE (Percentage)")
print("="*100)
print(decade_bin_pct.round(1))

In [ ]:
# Stacked bar chart by decade
fig, axes = plt.subplots(2, 1, figsize=(18, 14))

# Prepare data for plotting
x_pos = np.arange(len(decade_labels_list))
width = 0.7

# Plot 1: Stacked bar chart (counts)
bottom = np.zeros(len(decade_labels_list))
for i, bin_label in enumerate(bin_labels):
    values = decade_bin_crosstab[bin_label].values
    bars = axes[0].bar(x_pos, values, width, bottom=bottom, 
                       label=bin_label.split('\n')[0], 
                       color=colors_palette[i], edgecolor='black', linewidth=0.5)
    bottom += values

axes[0].set_xlabel('Decade', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Onset Date Bin Distribution by Decade (Stacked Counts)', fontsize=14, fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(decade_labels_list)
axes[0].legend(loc='upper left', bbox_to_anchor=(1.01, 1), fontsize=9)
axes[0].grid(True, alpha=0.3, axis='y')

# Plot 2: Stacked bar chart (percentages)
bottom = np.zeros(len(decade_labels_list))
for i, bin_label in enumerate(bin_labels):
    values = decade_bin_pct[bin_label].values
    bars = axes[1].bar(x_pos, values, width, bottom=bottom,
                       label=bin_label.split('\n')[0],
                       color=colors_palette[i], edgecolor='black', linewidth=0.5)
    
    # Add percentage labels for significant values
    for j, (bar, val) in enumerate(zip(bars, values)):
        if val > 5:  # Only label if > 5%
            axes[1].text(bar.get_x() + bar.get_width()/2., 
                        bottom[j] + val/2,
                        f'{val:.0f}%',
                        ha='center', va='center', fontsize=8, fontweight='bold')
    
    bottom += values

axes[1].set_xlabel('Decade', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Percentage (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Onset Date Bin Distribution by Decade (Stacked Percentages)', fontsize=14, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(decade_labels_list)
axes[1].legend(loc='upper left', bbox_to_anchor=(1.01, 1), fontsize=9)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_ylim([0, 100])

plt.tight_layout()
plt.show()

In [ ]:
# Grouped bar chart by decade (better for comparison across bins)
fig, ax = plt.subplots(figsize=(18, 8))

n_bins = len(bin_labels)
n_decades = len(decade_labels_list)
width = 0.8 / n_bins
x = np.arange(n_decades)

for i, bin_label in enumerate(bin_labels):
    offset = (i - n_bins/2) * width + width/2
    values = decade_bin_crosstab[bin_label].values
    bars = ax.bar(x + offset, values, width, label=bin_label.split('\n')[0], 
                  color=colors_palette[i], edgecolor='black', linewidth=0.5, alpha=0.9)

ax.set_xlabel('Decade', fontsize=12, fontweight='bold')
ax.set_ylabel('Count', fontsize=12, fontweight='bold')
ax.set_title('Onset Date Bin Distribution by Decade (Grouped Bars)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(decade_labels_list)
ax.legend(loc='upper left', bbox_to_anchor=(1.01, 1), fontsize=9)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 12.2 Binned Distribution by Data Split (Train/Validation/Test)

In [ ]:
# Binned distribution by data split
split_order = ['Train', 'Validation', 'Test']

# Create cross-tabulation
split_bin_crosstab = pd.crosstab(df['Split'], df['OnsetBin'])
split_bin_crosstab = split_bin_crosstab.reindex(index=split_order, columns=bin_labels, fill_value=0)

print("="*100)
print("ONSET DATE BIN DISTRIBUTION BY DATA SPLIT (Count)")
print("="*100)
print(split_bin_crosstab)

# Percentage within each split
split_bin_pct = pd.crosstab(df['Split'], df['OnsetBin'], normalize='index') * 100
split_bin_pct = split_bin_pct.reindex(index=split_order, columns=bin_labels, fill_value=0)

print("\n" + "="*100)
print("ONSET DATE BIN DISTRIBUTION BY DATA SPLIT (Percentage)")
print("="*100)
print(split_bin_pct.round(1))

In [ ]:
# Visualize split distributions
fig, axes = plt.subplots(2, 2, figsize=(20, 12))

split_colors = ['#ff9999', '#66b3ff', '#99ff99']

# Plot 1: Stacked bar chart by split (counts)
x_pos = np.arange(len(split_order))
width = 0.6

bottom = np.zeros(len(split_order))
for i, bin_label in enumerate(bin_labels):
    values = split_bin_crosstab[bin_label].values
    bars = axes[0, 0].bar(x_pos, values, width, bottom=bottom,
                          label=bin_label.split('\n')[0],
                          color=colors_palette[i], edgecolor='black', linewidth=0.5)
    bottom += values

axes[0, 0].set_xlabel('Data Split', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Onset Bin Distribution by Split (Stacked Counts)', fontsize=13, fontweight='bold')
axes[0, 0].set_xticks(x_pos)
axes[0, 0].set_xticklabels(split_order)
axes[0, 0].legend(loc='upper left', bbox_to_anchor=(1.01, 1), fontsize=8)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Plot 2: Stacked bar chart by split (percentages)
bottom = np.zeros(len(split_order))
for i, bin_label in enumerate(bin_labels):
    values = split_bin_pct[bin_label].values
    bars = axes[0, 1].bar(x_pos, values, width, bottom=bottom,
                          label=bin_label.split('\n')[0],
                          color=colors_palette[i], edgecolor='black', linewidth=0.5)
    
    # Add percentage labels
    for j, (bar, val) in enumerate(zip(bars, values)):
        if val > 5:
            axes[0, 1].text(bar.get_x() + bar.get_width()/2.,
                           bottom[j] + val/2,
                           f'{val:.0f}%',
                           ha='center', va='center', fontsize=9, fontweight='bold')
    
    bottom += values

axes[0, 1].set_xlabel('Data Split', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Percentage (%)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Onset Bin Distribution by Split (Stacked %)', fontsize=13, fontweight='bold')
axes[0, 1].set_xticks(x_pos)
axes[0, 1].set_xticklabels(split_order)
axes[0, 1].legend(loc='upper left', bbox_to_anchor=(1.01, 1), fontsize=8)
axes[0, 1].grid(True, alpha=0.3, axis='y')
axes[0, 1].set_ylim([0, 100])

# Plot 3: Grouped bar chart by split
n_bins = len(bin_labels)
n_splits = len(split_order)
width_grouped = 0.7 / n_bins
x = np.arange(n_splits)

for i, bin_label in enumerate(bin_labels):
    offset = (i - n_bins/2) * width_grouped + width_grouped/2
    values = split_bin_crosstab[bin_label].values
    bars = axes[1, 0].bar(x + offset, values, width_grouped,
                          label=bin_label.split('\n')[0],
                          color=colors_palette[i], edgecolor='black', linewidth=0.5, alpha=0.9)

axes[1, 0].set_xlabel('Data Split', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Onset Bin Distribution by Split (Grouped Bars)', fontsize=13, fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(split_order)
axes[1, 0].legend(loc='upper left', bbox_to_anchor=(1.01, 1), fontsize=8)
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Plot 4: Side-by-side comparison for each split (separate histograms)
for idx, split in enumerate(split_order):
    split_data = df[df['Split'] == split]['OnsetBin'].value_counts().reindex(bin_labels, fill_value=0)
    x_plot = np.arange(len(bin_labels))
    axes[1, 1].bar(x_plot + idx*0.25, split_data.values, width=0.24,
                   label=split, color=split_colors[idx], edgecolor='black',
                   linewidth=0.5, alpha=0.8)

axes[1, 1].set_xlabel('Onset Date Bins', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Onset Bin Comparison Across Splits', fontsize=13, fontweight='bold')
axes[1, 1].set_xticks(x_plot + 0.25)
axes[1, 1].set_xticklabels([label.split('\n')[0] for label in bin_labels], rotation=45, ha='right', fontsize=8)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 12.3 Heatmap Visualizations

In [ ]:
# Heatmap visualizations
fig, axes = plt.subplots(2, 1, figsize=(18, 14))

# Heatmap 1: Decade vs Bins (counts)
sns.heatmap(decade_bin_crosstab, annot=True, fmt='d', cmap='YlOrRd', 
            cbar_kws={'label': 'Count'}, linewidths=0.5, linecolor='gray',
            ax=axes[0])
axes[0].set_xlabel('Onset Date Bins', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Decade', fontsize=12, fontweight='bold')
axes[0].set_title('Heatmap: Onset Bin Frequency by Decade (Count)', fontsize=14, fontweight='bold')
axes[0].set_xticklabels([label.split('\n')[0] for label in bin_labels], rotation=45, ha='right')

# Heatmap 2: Decade vs Bins (percentages)
sns.heatmap(decade_bin_pct, annot=True, fmt='.1f', cmap='YlOrRd',
            cbar_kws={'label': 'Percentage (%)'}, linewidths=0.5, linecolor='gray',
            ax=axes[1], vmin=0, vmax=100)
axes[1].set_xlabel('Onset Date Bins', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Decade', fontsize=12, fontweight='bold')
axes[1].set_title('Heatmap: Onset Bin Frequency by Decade (Percentage)', fontsize=14, fontweight='bold')
axes[1].set_xticklabels([label.split('\n')[0] for label in bin_labels], rotation=45, ha='right')

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap for data splits
fig, axes = plt.subplots(2, 1, figsize=(18, 8))

# Heatmap 1: Split vs Bins (counts)
sns.heatmap(split_bin_crosstab, annot=True, fmt='d', cmap='RdYlGn_r',
            cbar_kws={'label': 'Count'}, linewidths=0.5, linecolor='gray',
            ax=axes[0])
axes[0].set_xlabel('Onset Date Bins', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Data Split', fontsize=12, fontweight='bold')
axes[0].set_title('Heatmap: Onset Bin Frequency by Data Split (Count)', fontsize=14, fontweight='bold')
axes[0].set_xticklabels([label.split('\n')[0] for label in bin_labels], rotation=45, ha='right')

# Heatmap 2: Split vs Bins (percentages)
sns.heatmap(split_bin_pct, annot=True, fmt='.1f', cmap='RdYlGn_r',
            cbar_kws={'label': 'Percentage (%)'}, linewidths=0.5, linecolor='gray',
            ax=axes[1], vmin=0, vmax=100)
axes[1].set_xlabel('Onset Date Bins', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Data Split', fontsize=12, fontweight='bold')
axes[1].set_title('Heatmap: Onset Bin Frequency by Data Split (Percentage)', fontsize=14, fontweight='bold')
axes[1].set_xticklabels([label.split('\n')[0] for label in bin_labels], rotation=45, ha='right')

plt.tight_layout()
plt.show()

### 12.4 Statistical Analysis of Binned Data

In [ ]:
# Statistical tests for bin distribution across splits
from scipy.stats import chi2_contingency

print("="*80)
print("CHI-SQUARE TEST FOR INDEPENDENCE")
print("="*80)

# Test 1: Split vs Bins
print("\n1. Testing if bin distribution is independent of data split:")
print("   H0: Bin distribution is independent of data split")
print("   H1: Bin distribution depends on data split")

chi2_split, p_split, dof_split, expected_split = chi2_contingency(split_bin_crosstab)

print(f"\n   Chi-square statistic: {chi2_split:.4f}")
print(f"   Degrees of freedom: {dof_split}")
print(f"   P-value: {p_split:.6f}")
print(f"   Result: {'REJECT H0' if p_split < 0.05 else 'FAIL TO REJECT H0'} (α=0.05)")

if p_split < 0.05:
    print("   → Bin distributions are significantly different across splits")
else:
    print("   → Bin distributions are similar across splits")

# Test 2: Decade vs Bins
print("\n2. Testing if bin distribution is independent of decade:")
print("   H0: Bin distribution is independent of decade")
print("   H1: Bin distribution depends on decade")

chi2_decade, p_decade, dof_decade, expected_decade = chi2_contingency(decade_bin_crosstab)

print(f"\n   Chi-square statistic: {chi2_decade:.4f}")
print(f"   Degrees of freedom: {dof_decade}")
print(f"   P-value: {p_decade:.6f}")
print(f"   Result: {'REJECT H0' if p_decade < 0.05 else 'FAIL TO REJECT H0'} (α=0.05)")

if p_decade < 0.05:
    print("   → Bin distributions have significantly changed over decades")
else:
    print("   → Bin distributions are consistent across decades")

print("\n" + "="*80)

In [ ]:
# Summary statistics for binned data
print("="*80)
print("SUMMARY: BINNED ANALYSIS KEY FINDINGS")
print("="*80)

print("\n1. OVERALL BIN DISTRIBUTION:")
bin_counts_sorted = df['OnsetBin'].value_counts().reindex(bin_labels)
most_common_bin = bin_counts_sorted.idxmax()
least_common_bin = bin_counts_sorted.idxmin()
print(f"   • Most common bin: {most_common_bin.split(chr(10))[0]} ({bin_counts_sorted.max()} occurrences)")
print(f"   • Least common bin: {least_common_bin.split(chr(10))[0]} ({bin_counts_sorted.min()} occurrences)")
print(f"   • Modal bin percentage: {bin_counts_sorted.max() / len(df) * 100:.1f}%")

print("\n2. DECADE PATTERNS:")
# Find which bin is most common in each decade
for decade_label in decade_labels_list:
    decade_row = decade_bin_crosstab.loc[decade_label]
    most_common = decade_row.idxmax()
    count = decade_row.max()
    percentage = count / decade_row.sum() * 100
    print(f"   • {decade_label}: {most_common.split(chr(10))[0]} ({count} samples, {percentage:.1f}%)")

print("\n3. DATA SPLIT PATTERNS:")
for split in split_order:
    split_row = split_bin_crosstab.loc[split]
    most_common = split_row.idxmax()
    count = split_row.max()
    percentage = count / split_row.sum() * 100
    print(f"   • {split:12s}: {most_common.split(chr(10))[0]} ({count} samples, {percentage:.1f}%)")

print("\n4. DISTRIBUTION BALANCE:")
# Calculate entropy or evenness of distribution
overall_pct = df['OnsetBin'].value_counts(normalize=True)
entropy = -np.sum(overall_pct * np.log(overall_pct + 1e-10))
max_entropy = np.log(len(bin_labels))
evenness = entropy / max_entropy
print(f"   • Distribution evenness: {evenness:.3f} (1.0 = perfectly even)")
print(f"   • Shannon entropy: {entropy:.3f} (max: {max_entropy:.3f})")

print("\n5. STATISTICAL SIGNIFICANCE:")
print(f"   • Split-wise difference: {'Significant' if p_split < 0.05 else 'Not significant'} (p={p_split:.4f})")
print(f"   • Decade-wise change: {'Significant' if p_decade < 0.05 else 'Not significant'} (p={p_decade:.4f})")

print("\n6. BIN STATISTICS:")
for bin_label in bin_labels:
    bin_data = df[df['OnsetBin'] == bin_label]['DateRelJun01']
    if len(bin_data) > 0:
        print(f"   • {bin_label.split(chr(10))[0]:20s}: n={len(bin_data):3d}, "
              f"mean={bin_data.mean():6.2f}d, std={bin_data.std():5.2f}d")

print("\n" + "="*80)